In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
## Importing necessary 


import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
## defining the path of our folder containing 10 folders with images corresponding to each category

data_dir1 = '../input/state-farm-distracted-driver-detection/imgs/train'
data_dir = pathlib.Path(data_dir1)

## image count

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

## Preprocessing


In [3]:

## defining batch size,size of image and dividing data into train and validation data with 20 % validation dataset

batch_size = 64
img_width = 256
img_height = 256

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  shuffle=True,
  image_size=(img_height, img_width),
  batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  shuffle=True,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

## defining class names 
train_ds.class_names = ['safe_drive', 'text_r', 'phone_r', 'text_l', 'phone_l', 'radio', 'drink', 'reach_bhd', 'hair_mkup', 'talk_passenger']
val_ds.class_names = ['safe_drive', 'text_r', 'phone_r', 'text_l', 'phone_l', 'radio', 'drink', 'reach_bhd', 'hair_mkup', 'talk_passenger']
class_names = train_ds.class_names
print(class_names)

In [4]:
## printing the images for different categories


plt.figure(figsize=(25, 25))
for images, labels in train_ds.take(3):
    
    for i in range(9):
        
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

for image_batch, labels_batch in train_ds:
  
    print(image_batch.shape)
    print(labels_batch.shape)
    break

### Image Augmentation


In [5]:

## image Augmentation which rotates,zooms,flips images to improve the accuracy of model

img_augmentation = Sequential(
    [
        layers.RandomTranslation(height_factor=0.1, width_factor=(0,0.1), input_shape=(img_height, img_width, 3)),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.1),
        layers.RandomZoom(-0.2, 0.1),
        layers.RandomContrast(factor=(0.2,0)),
    ],
    name="img_augmentation",
)


In [6]:
# An image after augmentation
plt.figure(figsize=(25, 25))
for image, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        aug_img = img_augmentation(tf.expand_dims(image[0], axis=0))
        plt.imshow(aug_img[0].numpy().astype("uint8"))
        plt.axis("off")

## Defining custom CNN model

In [31]:


## self defined Cnn model 

num_classes = len(class_names)

model = Sequential([

    img_augmentation,
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),  ## Rescaling pixel value 
    
    ## Adding CNN Layers 
    
    layers.Conv2D(16, 3, padding='same'),

    layers.Activation('relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(32, 3, padding='same'),

    layers.Activation('relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, padding='same'),

    layers.Activation('relu'),
    layers.MaxPooling2D(),
    
    ## Adding our main layers(Dense and output layer)
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax') ## Output layer
])
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['sparse_categorical_accuracy'])
model.summary()

In [32]:
## Fitting our data to the model
epochs=10  ## Only 10 epochs due to low computational power
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [33]:
model.evaluate(x=val_ds)  ## An accuracy of 81 % for validation data

## Using Transfer Learning with MobileNetV2 model as base model


In [22]:
from tensorflow.keras.applications import MobileNetV2  ## importing MobileNetV2 model

In [30]:
## Defining our model along with MobileNetV2


conv_model = MobileNetV2(weights='imagenet',include_top=False,input_shape = (256,256,3))


## Making last layer of Mobilent as Non trainable so that we can have our own output layer


for layer in conv_model.layers[:-3]:
    layer.trainable=False        


m_model = Sequential()
m_model.add(conv_model)
m_model.add(layers.Flatten())




m_model.add(layers.Dense(256,activation = 'relu'))


m_model.add(layers.Dense(128,activation = 'relu'))


m_model.add(layers.Dense(10,activation = 'softmax',name = 'output'))



m_model.summary()

In [24]:
m_model.compile(optimizer = optimizers.Adam(learning_rate=.0001) ,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['sparse_categorical_accuracy'])

In [28]:
## Running our model
epochs=10
history = m_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
)

## Final Accuracy


In [29]:
m_model.evaluate(x=val_ds)  ## Achieving a high accuracy of 96.7 % for validation data